# Размещения. Минимизиуруем суммарную длину проводов на плате

In [ ]:
import pandas as pd
import math
import copy
# Устанавливаем максимально возможную ширину вывода
pd.set_option('display.width', None)  # или большое число, например 200
pd.set_option('display.max_columns', None)  # показывать все столбцы
pd.set_option('display.expand_frame_repr', False)  # не переносить строки

wires = [  # Вариант 15
    [0, 0, 1, 1, 3, 1, 0, 0, 0, 2, 0, 0, 0, 0, 0, 1, 1, 4, 0, 0],
    [0, 0, 0, 1, 1, 1, 0, 3, 2, 1, 0, 0, 0, 1, 0, 3, 0, 0, 1, 0],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 2, 0, 0, 3, 0, 0, 1, 0, 0],
    [1, 1, 0, 0, 0, 1, 4, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1],
    [3, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0],
    [1, 1, 0, 1, 0, 0, 0, 3, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 2, 3],
    [0, 0, 0, 4, 0, 0, 0, 0, 0, 1, 1, 2, 1, 0, 2, 0, 0, 0, 0, 0],
    [0, 3, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 3, 1, 0, 0, 0, 1, 1],
    [0, 2, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 3, 1, 0, 1, 2, 0, 0],
    [2, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 4, 1, 0, 0, 0],
    [0, 0, 3, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1],
    [0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2, 0, 2, 0, 1, 0, 0, 1],
    [0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 2, 0, 1, 1, 0, 1, 0, 1, 1],
    [0, 1, 0, 0, 0, 1, 0, 3, 3, 0, 1, 0, 1, 0, 0, 0, 0, 0, 2, 2],
    [0, 0, 3, 0, 0, 0, 2, 1, 1, 1, 1, 2, 1, 0, 0, 2, 2, 0, 0, 0],
    [1, 3, 0, 0, 2, 0, 0, 0, 0, 4, 1, 0, 0, 0, 2, 0, 0, 1, 1, 1],
    [1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 2, 0, 0, 0, 0, 0],
    [4, 0, 1, 1, 0, 0, 0, 0, 2, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 2, 0, 1, 0, 0, 0, 0, 1, 2, 0, 1, 0, 0, 0, 2],
    [0, 0, 0, 1, 0, 3, 0, 1, 0, 0, 1, 1, 1, 2, 0, 1, 0, 0, 2, 0]
]
# Маленький пример для тестирования
# b1 = [
#     [0, 1, 2, 3, 4],
#     [5, 6, 7, 8, 9],
#     [10, 11, 12, 13, 14],
#     [15, 16, 17, 18, 19]
# ]

# b2 = [
#     [0, -1, 2],
#     [5, 6, -1]
# ]



## Первая часть. Формируем хоть какое-то решение

In [6]:
def p(n):
    # n - номер вершины
    return sum(wires[n])


def get_position_by_node(n, board):
    # По номеру вершины и плате получает координаты вершины.
    for i, row in enumerate(board):
        for j, column in enumerate(row):
            element = board[i][j]  # Номер вершины
            if element == n:
                return (i, j)



def get_Q_for_node(n, board):
    # По номеру вершины и плате получает оценку размещения этой вершины.
    # n - номер вершины
    # board - состояние платы
    sum = 0
    if n == -1:
        return 0
    for element, s in enumerate(wires[n]):
        n_x, n_y = get_position_by_node(n, board)
        # print(n_x, n_y)
        if get_position_by_node(element, board) is None:
            continue
        element_x, element_y = get_position_by_node(element, board)
        sum += s * (abs(element_x - n_x) + abs(element_y - n_y))

    return sum



def Q(board):
    sum = 0
    for i, row in enumerate(board):
        for j, column in enumerate(row):
            sum += get_Q_for_node(board[i][j], board)
    return sum




################# Ищем начальное приближение ######################


board = [
    [-1, -1, -1, -1, -1],
    [-1, -1, -1, -1, -1],
    [-1, -1, -1, -1, -1],
    [-1, -1, -1, -1, -1]
]

print("Матрица до нулевого размещения")
print(pd.DataFrame(board))

board[0][0] = 0
placed_nodes = [0]

print("Матрица после нулевого размещения")
print(pd.DataFrame(board))
 
for iteration in range(len(wires) - 1):  # Создаем новый k столбик. Размещаем новую вершину.
    # Считаем самый большой k
    print(f"==================== Этап {iteration} ==========================")
    k_max = -math.inf
    n_with_k_max = 0
    for n, row in enumerate(wires):
        if n in placed_nodes:
            continue
        s_placed = 0
        for placed_node in placed_nodes:
            s_placed += wires[placed_node][n]
        k = 2 * s_placed - p(n)
        if k > k_max:
            k_max = k
            n_with_k_max = n
    print(f"Максимальный k = {k} получен в вершине {n_with_k_max}")
    # Выбираем, куда расположить вершину с максимальным k
    Q_min = math.inf
    i_with_Q_min = 0
    j_with_Q_min = 0
    for i, row in enumerate(board):
        for j, column in enumerate(row):
            element = board[i][j]  # Номер вершины
            if element in placed_nodes:
                continue
            possible_board = copy.deepcopy(board)
            possible_board[i][j] = n_with_k_max
            # print(f"possible_board: {possible_board}")
            possible_Q = Q(possible_board)
            # print(f"possible_Q: {possible_Q}")
            if possible_Q < Q_min:
                Q_min = possible_Q
                i_with_Q_min = i
                j_with_Q_min = j
    board[i_with_Q_min][j_with_Q_min] = n_with_k_max
    placed_nodes.append(n_with_k_max)
    print(f"Полученная на этапе {iteration} матрица ({n_with_k_max} размещена в ({i_with_Q_min}, {j_with_Q_min}))")
    print(pd.DataFrame(board))


Матрица до нулевого размещения
   0  1  2  3  4
0 -1 -1 -1 -1 -1
1 -1 -1 -1 -1 -1
2 -1 -1 -1 -1 -1
3 -1 -1 -1 -1 -1
Матрица после нулевого размещения
   0  1  2  3  4
0  0 -1 -1 -1 -1
1 -1 -1 -1 -1 -1
2 -1 -1 -1 -1 -1
3 -1 -1 -1 -1 -1
==================== Этап 0 ==========================
Максимальный k = -13 получен в вершине 4
Полученная на этапе 0 матрица (4 размещена в (0, 1))
   0  1  2  3  4
0  0  4 -1 -1 -1
1 -1 -1 -1 -1 -1
2 -1 -1 -1 -1 -1
3 -1 -1 -1 -1 -1
==================== Этап 1 ==========================
Максимальный k = -13 получен в вершине 17
Полученная на этапе 1 матрица (17 размещена в (1, 0))
    0  1  2  3  4
0   0  4 -1 -1 -1
1  17 -1 -1 -1 -1
2  -1 -1 -1 -1 -1
3  -1 -1 -1 -1 -1
==================== Этап 2 ==========================
Максимальный k = -13 получен в вершине 2
Полученная на этапе 2 матрица (2 размещена в (1, 1))
    0  1  2  3  4
0   0  4 -1 -1 -1
1  17  2 -1 -1 -1
2  -1 -1 -1 -1 -1
3  -1 -1 -1 -1 -1
==================== Этап 3 =======================

## Итерационное улучшение. Доводим решение до идеала

In [7]:
def get_worst_node(board):
    worst_node = 0
    worst_Q_for_wire = -math.inf
    for i, row in enumerate(board):
        for j, column in enumerate(row):
            element = board[i][j]  # Номер вершины
            Q_for_wire = get_Q_for_node(element, board) / p(element)
            if Q_for_wire > worst_Q_for_wire:
                worst_Q_for_wire = Q_for_wire
                worst_node = element
    return worst_node, worst_Q_for_wire



while True:
    print(pd.DataFrame(board))
    worst_node, worst_Q_for_wire = get_worst_node(board)
    print(f"Худшим размещением признана вершина {worst_node} с соотношением Q за провод = {worst_Q_for_wire}")

    # Ищём "центр тяжести вершины" (То, куда в среднем уходят от неё провода)
    x_center = 0
    y_center = 0
    for node, s in enumerate(wires[worst_node]):  # Идём по одной вершине
        if s == 0:
            continue
        x_node, y_node = get_position_by_node(node, board)
        x_center += s * x_node
        y_center += s * y_node
    x_center /= p(worst_node)
    y_center /= p(worst_node)
    print(f"Центр масс этой вершины - ({x_center}, {y_center})")

    possible_x = [math.floor(x_center), math.ceil(x_center)]
    possible_y = [math.floor(x_center), math.ceil(y_center)]

    Q_min = math.inf
    more_optimal_board = copy.deepcopy(board)
    swapped_element = 0
    for x in possible_x:
        for y in possible_y:
            worst_node_x, worst_node_y = get_position_by_node(worst_node, board)
            possible_board = copy.deepcopy(board)
            possible_board[worst_node_x][worst_node_y] = board[x][y]
            possible_board[x][y] = worst_node
            possible_Q = Q(possible_board)
            # print(f"Пробуем обмени с {board[x][y]} {possible_board}")
            # print(possible_Q)
            if possible_Q < Q_min:
                Q_min = possible_Q
                swapped_element = board[x][y]
                more_optimal_board = copy.deepcopy(possible_board)

    print(f"Наиболее эффективным получился обмен вершин {worst_node} и {swapped_element}")
    print(f"Q: {Q(board)} -> {Q(more_optimal_board)}")

    if Q(board) <= Q(more_optimal_board):
        print("==== Процесс оптимизации завершен =====")
        print(f"Итоговый оптимальный вариант с Q = {Q(board)}")
        print(pd.DataFrame(board))
        break

    board = copy.deepcopy(more_optimal_board)







    0   1   2   3   4
0   0   4   9  15   1
1  17   2  10   3   5
2  16  11  14   6   7
3   8  12  13  19  18
Худшим размещением признана вершина 8 с соотношением Q за провод = 3.3846153846153846
Центр масс этой вершины - (1.4615384615384615, 1.8461538461538463)
Наиболее эффективным получился обмен вершин 8 и 10
Q: 490 -> 486
    0   1   2   3   4
0   0   4   9  15   1
1  17   2   8   3   5
2  16  11  14   6   7
3  10  12  13  19  18
Худшим размещением признана вершина 10 с соотношением Q за провод = 3.2222222222222223
Центр масс этой вершины - (1.5555555555555556, 1.7777777777777777)
Наиболее эффективным получился обмен вершин 10 и 11
Q: 486 -> 486
==== Процесс оптимизации завершен =====
Итоговый оптимальный вариант с Q = 486
    0   1   2   3   4
0   0   4   9  15   1
1  17   2   8   3   5
2  16  11  14   6   7
3  10  12  13  19  18
